In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import transect_viz
from transect_viz import transect_cdec_data
from transect_viz import transect_generator
import hvplot.pandas

In [ ]:
df=transect_viz.load_transect_file('../data/Transect_20220302_Fabian.csv')
df['EC'] = df['Sonde1_SpCond'] # 'EC' is mapped from Sonde1_SpCond
df.head(3)

In [ ]:
import panel as pn
pn.extension()

import holoviews as hv

In [ ]:
def map_with_legend_by_color_size(df, cmap='rainbow4'):
    mapcz, legend = transect_viz.map_transect_with_size_and_color(df, cmap=cmap)
    return mapcz, legend

In [ ]:
#list(hv.element.tile_sources.keys()) # full_list
tiles=pn.widgets.Select(options=['CartoLight','StamenTerrain','OSM','EsriImagery'], value='CartoLight')

In [ ]:
#from holoviews.plotting import list_cmaps
#list_cmaps(category='Uniform Sequential')
#list_cmaps()

cmap_list=['rainbow4','inferno','viridis','ColorBlind']
cmaps=pn.widgets.Select(options=cmap_list, value='rainbow4')

In [ ]:
#mapcs = pn.Column(pn.Column(tiles,cmaps), pn.bind(map_by_color_size, df=df, tile_layer=tiles, cmap=cmaps))
#mapcs 

In [ ]:
cdec_stations=['ORM', 'ODM', 'OLD', 'GLE', 'GLC']
station_info_file='../data/station_info_file.csv'

In [ ]:
dfs = transect_cdec_data.get_stations_cached(cdec_stations)
dfs = transect_viz.add_in_station_info(dfs, station_info_file=station_info_file)

In [ ]:
def create_transect_profile(df):
    gdf = transect_generator.to_geoframe(df)
    transect_line = transect_generator.create_transect_line(gdf)
    gdf = transect_generator.add_transect_dist(gdf, transect_line)
    return gdf.hvplot.scatter(x='transect_dist', y='EC').opts(
        title='Profile of Specific Conductivity along Transect ', xlabel='Transect Distance (ft)', ylabel='Specific Conductivity')

In [ ]:
transect_profile_plot = create_transect_profile(df)
transect_profile_plot

In [ ]:
station_map_with_tiles = transect_viz.get_tile_layer('CartoLight') *\
    transect_viz.points_for_station(dfs) *\
    transect_viz.labels_for_stations(dfs).opts(frame_height=350)

In [ ]:
sdate, edate = transect_viz.get_start_end_dates(df)
bsdate, bedate = transect_viz.get_buffered_start_end_dates(df)

In [ ]:
def make_list(df):
    return [df[[c]] for c in df.columns]

In [ ]:
cdec_flows = transect_cdec_data.get_cdec_data_cached(bsdate, bedate, cdec_stations, data_type='flow')
cdec_flows = make_list(cdec_flows)

In [ ]:
cdec_vels = transect_cdec_data.get_cdec_data_cached(bsdate, bedate, cdec_stations, data_type='vel')
cdec_vels = make_list(cdec_vels)

In [ ]:
collection_span = transect_viz.timespan(sdate, edate)

In [ ]:
flow_plot = transect_viz.plot_flow_around_time(sdate,edate,cdec_flows)*collection_span

In [ ]:
velocity_plot = transect_viz.plot_velocity_around_time(sdate,edate,cdec_vels)*collection_span

In [ ]:
from holoviews import opts

In [ ]:
dftidalvelgis = transect_viz.merge_mean_velocity_with_stations(dfs, transect_viz.mean_velocity_over_time(sdate,edate,transect_viz.get_tidal_filtered(cdec_vels)))

In [ ]:
vel_tidal_vectors = transect_viz.plot_velocity_vectors(dftidalvelgis)*transect_viz.plot_velocity_labels(dftidalvelgis)

In [ ]:
vel_tidal_map_with_station = station_map_with_tiles*vel_tidal_vectors
vel_tidal_map_with_station.opts(frame_height=350, title='Mean Tidally filtered Velocity over collection period', xlabel='Longitude', ylabel='Latitude')

In [ ]:
dfvelgis = transect_viz.merge_mean_velocity_with_stations(dfs, transect_viz.mean_velocity_over_time(sdate,edate,cdec_vels))

In [ ]:
vel_vectors = transect_viz.plot_velocity_vectors(dfvelgis)*transect_viz.plot_velocity_labels(dfvelgis)

In [ ]:
#ymin, ymax = hv.util.transform.lon_lat_to_easting_northing(-121.5,37.78)[1], hv.util.transform.lon_lat_to_easting_northing(-121.5,37.83)[1]
#ylim=(ymin,ymax))

In [ ]:
vel_map_with_station = station_map_with_tiles*vel_vectors
vel_map_with_station.opts(frame_height=350, title='Mean Velocity over collection period', xlabel='Longitude', ylabel='Latitude')

In [ ]:
def show_begin_end_pts_with_labels(df):
    begin_end = df.iloc[[0, -1]][['Longitude', 'Latitude', 'DateTime']].copy()
    begin_end['Label'] = ['Start', 'End']
    print(begin_end)
    plt = begin_end.iloc[0:1, :].hvplot.labels(
        x='Longitude', y='Latitude', text='Label', geo=True).opts(opts.Labels(yoffset=300))
    plt = plt * begin_end.iloc[-1:, :].hvplot.labels(
        x='Longitude', y='Latitude', text='Label', geo=True).opts(opts.Labels(yoffset=-300))
    plt = plt * begin_end.hvplot.points(geo=True)
    return plt

In [ ]:
vel_map_with_station*show_begin_end_pts_with_labels(df)

In [ ]:
map1, legend1 = map_with_legend_by_color_size(df)
map1 = transect_viz.get_tile_layer('CartoLight')*map1.opts(xlabel='Longitude', ylabel='Latitude')
map1 = map1.opts(opts.Points(colorbar=True)).opts(title='Transect EC by size and color')

In [ ]:
vel_map2=pn.Row(vel_map_with_station.opts(frame_width=700, frame_height=300), vel_tidal_map_with_station.opts(frame_width=700, frame_height=300))

In [ ]:
#map_panel = pn.Row(pn.Column(map1.opts(frame_width=700, frame_height=300),vel_map2),legend1.opts(title='Point Size Legend'))
map_panel = pn.Row(pn.Column(pn.Row(map1.opts(frame_width=700, frame_height=300),legend1.opts(title='Point Size Legend')),
                             pn.Row(vel_map_with_station.opts(frame_width=700, frame_height=300),
                                   transect_profile_plot)))

In [ ]:
#list(hv.element.tile_sources.keys()) # full_list
tiles=pn.widgets.Select(options=['CartoLight','StamenTerrain','OSM','EsriImagery'], value='CartoLight')

In [ ]:
#from holoviews.plotting import list_cmaps
#list_cmaps(category='Uniform Sequential')
#list_cmaps()

cmap_list=['rainbow4','inferno','viridis','ColorBlind']
cmaps=pn.widgets.Select(options=cmap_list, value='rainbow4')

In [ ]:
#mapcs = pn.Column(pn.Column(tiles,cmaps), pn.bind(map_by_color_size, df=df, tile_layer=tiles, cmap=cmaps))
#mapcs 

In [ ]:
tidal_plot = transect_viz.plot_tidal_filtered_flow_around_time(sdate, edate, transect_viz.get_tidal_filtered(cdec_flows))*collection_span

In [ ]:
tidal_vel_plot = transect_viz.plot_tidal_filtered_velocity_around_time(sdate, edate, transect_viz.get_tidal_filtered(cdec_vels))*collection_span

In [ ]:
flow_velocity_panel = pn.Column(pn.Row(flow_plot, tidal_plot), pn.Row(velocity_plot, tidal_vel_plot))

In [ ]:
cdec_stage = transect_cdec_data.get_cdec_data_cached(
    bsdate, bedate, cdec_stations, data_type='stage')
cdec_stage = make_list(cdec_stage)
cdec_ec = transect_cdec_data.get_cdec_data_cached(
    bsdate, bedate, cdec_stations, data_type='ec')
cdec_ec = make_list(cdec_ec)

stage_plot = transect_viz.plot_data_around_time(
    sdate, edate, cdec_stage, data_type='stage', ylabel='Stage (ft)', title='Stage')*transect_viz.timespan(sdate, edate)

tidal_stage_plot = transect_viz.plot_data_around_time(sdate, edate, transect_viz.get_tidal_filtered(
    cdec_stage), data_type='stage', ylabel='Stage (ft)', title='Stage tidally filtered')*transect_viz.timespan(sdate, edate)

ec_plot = transect_viz.plot_data_around_time(
    sdate, edate, cdec_ec, data_type='ec', ylabel='EC (umhos/cm)', title='EC')*transect_viz.timespan(sdate, edate)

tidal_ec_plot = transect_viz.plot_data_around_time(sdate, edate, transect_viz.get_tidal_filtered(
    cdec_ec), data_type='ec', ylabel='Tidally filtered EC (umhos/cm)', title='EC')*transect_viz.timespan(sdate, edate)

In [ ]:
stage_ec_panel = pn.Column(pn.Row(stage_plot, tidal_stage_plot), pn.Row(ec_plot, tidal_ec_plot))

In [ ]:
full_panel = pn.Column(map_panel, flow_velocity_panel, stage_ec_panel)

In [ ]:
full_panel

In [ ]:
pn.io.save.save(full_panel,'../transect_reports/Transect_Viz_20220302_Fabian.html',title='Fabian Tract 2022-03-02 EC Transect Visualization Panel')